In [ ]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import string
import json
import warnings
warnings.filterwarnings(action = 'ignore')
import itertools
import numpy as np
import math
from numpy import dot
from numpy.linalg import norm

#import nltk
#nltk.download('punkt')
#from nltk.tokenize import sent_tokenize, word_tokenize

!pip install datasets
from datasets import Dataset

**BERT**

In [ ]:
############################## Read Data ##############################
#######################################################################
# read data
df_train1 = pd.read_excel("/content/drive/MyDrive/sample_data.xlsx")
df_train1.reset_index(drop=True, inplace=True)

df_train2 = Dataset.from_pandas(df_train1)
df_train2 = df_train2.rename_column('contributor','label')
df_train2


In [ ]:
############################## Train Model ##############################
#########################################################################
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# model selection
Tokenizer_name = "bert-base-uncased"
Model_name = "bert-base-uncased"
Number_of_labels = 2

tokenizer = AutoTokenizer.from_pretrained(Tokenizer_name)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = df_train2.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['sentence'])

# set up stratified 5-fold sampling
label_for_s_kfold = tokenized_datasets['label']
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from transformers import TrainingArguments

from datasets import load_metric
training_args = TrainingArguments("test_trainer")

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import Trainer

s_kfold = StratifiedKFold(n_splits=5, random_state=None, shuffle=False) # choose n
my_accuracy = []
my_precison = []
my_recall = []
my_fscore = []
my_confusion_matrix = []

for train_index, test_index in s_kfold.split(np.zeros(len(label_for_s_kfold)), label_for_s_kfold):
  my_train_set = tokenized_datasets.select(train_index)
  my_test_set = tokenized_datasets.select(test_index )

  model = AutoModelForSequenceClassification.from_pretrained(Model_name, num_labels = Number_of_labels)

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=my_train_set,
    eval_dataset=my_test_set,
    compute_metrics=compute_metrics,
)
  trainer.train()

  predictions = trainer.predict(my_test_set)
  array_predict = predictions.predictions
  list_actual = my_test_set['label']
  list_predict = []
  for i in range(array_predict.shape[0]):
    list_predict.append(np.argmax(array_predict[i]))

  my_precison_new, my_recall_new, my_fscore_new, my_support_new = precision_recall_fscore_support(list_actual, list_predict, average='weighted')

  my_accuracy.append(trainer.evaluate()['eval_accuracy'])

  my_precison.append(my_precison_new)
  my_recall.append(my_recall_new)
  my_fscore.append(my_fscore_new)

  my_confusion_matrix.append(confusion_matrix(list_actual,list_predict))

print("fscore:", sum(my_fscore)/len(my_fscore))
print("confusion_matrix:\n", sum(my_confusion_matrix) / len(my_confusion_matrix))


In [ ]:
############################## Predict #################################
########################################################################
df_test1 = pd.read_excel("/content/drive/MyDrive/your_data.xlsx")
df_test1.reset_index(drop=True, inplace=True)
df_test1 = df_test1[['sentence']]
df_test2 = Dataset.from_pandas(df_test1)

tokenized_datasets = df_test2.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['sentence'])
my_test_set = tokenized_datasets

# predict using trained model
predictions = trainer.predict(my_test_set)
array_predict = predictions.predictions

list_predict = []
for i in range(array_predict.shape[0]):
  list_predict.append(np.argmax(array_predict[i]))

df_test1["contributor"] = list_predict
df_test1

**RoBERTa**

In [ ]:
############################## Read Data ##############################
#######################################################################
# read data
df_train1 = pd.read_excel("/content/drive/MyDrive/sample_data.xlsx")
df_train1.reset_index(drop=True, inplace=True)

df_train2 = Dataset.from_pandas(df_train1)
df_train2 = df_train2.rename_column('contributor','label')
df_train2


In [ ]:
############################## Train Model ##############################
#########################################################################
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# model selection
Tokenizer_name = "roberta-base"
Model_name = "roberta-base"
Number_of_labels = 2

tokenizer = AutoTokenizer.from_pretrained(Tokenizer_name)

def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = df_train2.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['sentence'])

# set up stratified 5-fold sampling
label_for_s_kfold = tokenized_datasets['label']
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from transformers import TrainingArguments

from datasets import load_metric
training_args = TrainingArguments("test_trainer")

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import Trainer

s_kfold = StratifiedKFold(n_splits=5, random_state=None, shuffle=False) # choose n
my_accuracy = []
my_precison = []
my_recall = []
my_fscore = []
my_confusion_matrix = []

for train_index, test_index in s_kfold.split(np.zeros(len(label_for_s_kfold)), label_for_s_kfold):
  my_train_set = tokenized_datasets.select(train_index)
  my_test_set = tokenized_datasets.select(test_index )

  model = AutoModelForSequenceClassification.from_pretrained(Model_name, num_labels = Number_of_labels)

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=my_train_set,
    eval_dataset=my_test_set,
    compute_metrics=compute_metrics,
)
  trainer.train()

  predictions = trainer.predict(my_test_set)
  array_predict = predictions.predictions
  list_actual = my_test_set['label']
  list_predict = []
  for i in range(array_predict.shape[0]):
    list_predict.append(np.argmax(array_predict[i]))

  my_precison_new, my_recall_new, my_fscore_new, my_support_new = precision_recall_fscore_support(list_actual, list_predict, average='weighted')

  my_accuracy.append(trainer.evaluate()['eval_accuracy'])

  my_precison.append(my_precison_new)
  my_recall.append(my_recall_new)
  my_fscore.append(my_fscore_new)

  my_confusion_matrix.append(confusion_matrix(list_actual,list_predict))

print("fscore:", sum(my_fscore)/len(my_fscore))
print("confusion_matrix:\n", sum(my_confusion_matrix) / len(my_confusion_matrix))


In [ ]:
############################## Predict #################################
########################################################################
df_test1 = pd.read_excel("/content/drive/MyDrive/your_data.xlsx")
df_test1.reset_index(drop=True, inplace=True)
df_test1 = df_test1[['sentence']]
df_test2 = Dataset.from_pandas(df_test1)

tokenized_datasets = df_test2.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['sentence'])
my_test_set = tokenized_datasets

# predict using trained model
predictions = trainer.predict(my_test_set)
array_predict = predictions.predictions

list_predict = []
for i in range(array_predict.shape[0]):
  list_predict.append(np.argmax(array_predict[i]))

df_test1["contributor"] = list_predict
df_test1

**GPT-2**

In [ ]:
############################## Read Data ##############################
#######################################################################
# read data
df_train1 = pd.read_excel("/content/drive/MyDrive/sample_data.xlsx")
df_train1.reset_index(drop=True, inplace=True)

df_train2 = Dataset.from_pandas(df_train1)
df_train2 = df_train2.rename_column('contributor','label')
df_train2

In [ ]:
############################## Train Model ##############################
#########################################################################
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# model selection
Tokenizer_name = "gpt2"
Model_name = "gpt2"
Number_of_labels = 2

#pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(Tokenizer_name)

tokenizer.pad_token = tokenizer.eos_token
def tokenize_function(examples):
    return tokenizer(examples["sentence"],padding="max_length", truncation=True, max_length = 57)

tokenized_datasets = df_train2.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(['sentence'])

# set up stratified 5-fold sampling
label_for_s_kfold = tokenized_datasets['label']
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from transformers import TrainingArguments

from datasets import load_metric
training_args = TrainingArguments("test_trainer")

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import Trainer


s_kfold = StratifiedKFold(n_splits=5, random_state=None, shuffle=False) #set k =5
my_accuracy = []
my_precison = []
my_recall = []
my_fscore = []
my_confusion_matrix = []

for train_index, test_index in s_kfold.split(np.zeros(len(label_for_s_kfold)), label_for_s_kfold):
  my_train_set = tokenized_datasets.select(train_index)
  my_test_set = tokenized_datasets.select(test_index )

  model = AutoModelForSequenceClassification.from_pretrained(Model_name, num_labels = Number_of_labels)
  model.config.pad_token_id = model.config.eos_token_id

  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=my_train_set,
    eval_dataset=my_test_set,
    compute_metrics=compute_metrics,
)
  trainer.train()

  predictions = trainer.predict(my_test_set)
  array_predict = predictions.predictions
  list_actual = my_test_set['label']
  list_predict = []
  for i in range(array_predict.shape[0]):
    list_predict.append(np.argmax(array_predict[i]))

  my_precison_new, my_recall_new, my_fscore_new, my_support_new = precision_recall_fscore_support(list_actual, list_predict, average='weighted')

  my_accuracy.append(trainer.evaluate()['eval_accuracy'])

  my_precison.append(my_precison_new)
  my_recall.append(my_recall_new)
  my_fscore.append(my_fscore_new)

  my_confusion_matrix.append(confusion_matrix(list_actual,list_predict))

print("fscore:", sum(my_fscore)/len(my_fscore))
print("confusion_matrix:\n", sum(my_confusion_matrix) / len(my_confusion_matrix))

In [ ]:
############################## Predict #################################
########################################################################
df_test1 = pd.read_excel("/content/drive/MyDrive/your_data.xlsx")
df_test1.reset_index(drop=True, inplace=True)
df_test1 = df_test1[['sentence']]
df_test2 = Dataset.from_pandas(df_test1)

tokenized_datasets = df_test2.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['sentence'])
my_test_set = tokenized_datasets

# predict using trained model
predictions = trainer.predict(my_test_set)
array_predict = predictions.predictions

list_predict = []
for i in range(array_predict.shape[0]):
  list_predict.append(np.argmax(array_predict[i]))

df_test1["contributor"] = list_predict
df_test1

**GPT-3**

In [ ]:
############################## Read Data ##############################
#######################################################################
# Enter credentials
import openai
%env OPENAI_API_KEY = your_key

# read data
df_train1 = pd.read_excel("/content/drive/MyDrive/sample_data.xlsx")
df_train1.reset_index(drop=True, inplace=True)

df_train1.columns = ['prompt','completion']
df_train1.to_json("/content/drive/MyDrive/df_train_final.jsonl", orient='records', lines=True)
!openai tools fine_tunes.prepare_data -f "/content/drive/MyDrive/df_train_final.jsonl"

In [ ]:
############################## Train Model ##############################
#########################################################################
!openai api fine_tunes.create \
    -t "/content/drive/MyDrive/df_train_final_prepared_train.jsonl" \
    -v "/content/drive/MyDrive/df_train_final_prepared_valid.jsonl" \
    --compute_classification_metrics \
    --classification_positive_class \
    " 0" \
    -m ada \ # model selection
    #--n_epochs 4 \
    #--batch_size $batch_size \
    #--learning_rate_multiplier $learning_rate_multiplier \
    #--prompt_loss_weight $prompt_loss_weight

In [ ]:
# continue fine-tune
!openai api fine_tunes.follow -i your_model_id

In [ ]:
#save results
!openai api fine_tunes.results -i ft-uIXlcKUcurKy9uMjUCsV5s7G > "/content/drive/MyDrive/GPT3_ft/df_train_final_done.csv"
results = pd.read_csv("/content/drive/MyDrive/GPT3_ft/df_train_final_done.csv")
results[results['classification/accuracy'].notnull()].tail(1)

In [ ]:
############################## Predict #################################
########################################################################
# set up your model
openai.api_key ="your_key"
ft_model = 'your_model'

# divide large data into small parts and predict
df_test = pd.read_excel("/content/drive/MyDrive/your_data.xlsx")
df_test.reset_index(drop=True, inplace=True)
df_test = df_test[['sentence']]



div_obs = 5000
last_part_n_saw = 10 # set up starting point

for i in range(last_part_n_saw, df_test.shape[0]//div_obs + 1):
  # index & corpus subsample

  df_test_sub = df_test[i*div_obs:(i+1)*div_obs]
  df_test_sub.reset_index(drop=True, inplace=True)

  list_predict = []
  for index,row in df_test_sub.iterrows():
    my_prompt = df_test_sub['sentence'][index] + " ->"

    res = openai.Completion.create(model=ft_model, prompt=my_prompt, max_tokens=1, temperature=0)

    list_predict.append(res['choices'][0]['text'])

  df_test["contributor"] = list_predict
  df_test.to_csv("/content/drive/MyDrive/your_data_part"+str(i + 1)+".csv",index = False)


# append all parts
df_test_all = pd.read_csv("/content/drive/MyDrive/your_data_part1.csv")
for i in range(1, df_test.shape[0]//div_obs + 1):
  df_test_all_next = pd.read_csv("/content/drive/MyDrive/your_data_part"+str(i + 1)+".csv")
  df_test_all = df_test_all.append(df_test_all_next)

df_test_all